In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [4]:
data = pd.read_csv('merged_data.csv')


In [12]:
data

,Index,Sno,Movie Name,Director1,Director2,Duration,Year,Ratings,Metascore,Star1,Star2,Star3,Star4,Votes,Genre1,Genre2,Genre3,Gross Collection,Popularity,Certification
0,1,1,The Dark Knight,Christopher Nolan,Christian Bale,152,2008,9.0,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,2737930,Action,Crime,Drama,2737930,NaN,UA
1,2,2,The Lord of the Rings: The Return of the King,Peter Jackson,Elijah Wood,201,2003,9.0,94.0,Peter Jackson,Elijah Wood,Viggo Mortensen,Ian McKellen,1897456,Action,Adventure,Drama,1897456,NaN,U
2,3,3,Spider-Man: Across the Spider-Verse,Joaquim Dos Santos,Kemp Powers,140,2023,8.9,86.0,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,Shameik Moore,179975,Animation,Action,Adventure,179975,NaN,U
3,4,4,Inception,Christopher Nolan,Leonardo DiCaprio,148,2010,8.8,74.0,Christopher Nolan,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,2429596,Action,Adventure,Sci-Fi,2429596,NaN,UA
4,5,5,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,Elijah Wood,178,2001,8.8,92.0,Peter Jackson,Elijah Wood,Ian McKellen,Orlando Bloom,1925886,Action,Adventure,Drama,1925886,NaN,U
5,6,6,The Lord of the Rings: The Two Towers,Peter Jackson,Elijah Wood,179,2002,8.8,87.0,Peter Jackson,Elijah Wood,Ian McKellen,Viggo Mortensen,1712386,Action,Adventure,Drama,1712386,NaN,UA
6,7,7,The Matrix,Lana Wachowski,Lilly Wachowski,136,1999,8.7,73.0,Lana Wachowski,Lilly Wachowski,Keanu Reeves,Laurence Fishburne,1969511,Action,Sci-Fi,NaN,1969511,NaN,A
7,8,8,Star Wars: Episode V - The Empire Strikes Back,Irvin Kershner,Mark Hamill,124,1980,8.7,82.0,Irvin Kershner,Mark Hamill,Harrison Ford,Carrie Fisher,1328745,Action,Adventure,Fantasy,1328745,NaN,UA
8,9,9,Soorarai Pottru,Sudha Kongara,Suriya,153,2020,8.7,NaN,Sudha Kongara,Suriya,Paresh Rawal,Aparna Balamurali,119962,Action,Drama,NaN,119962,NaN,U
9,10,10,Terminator 2: Judgment Day,James Cameron,Arnold Schwarzenegger,137,1991,8.6,75.0,James Cameron,Arnold Schwarzenegger,Linda Hamilton,Edward Furlong,1130058,Action,Sci-Fi,NaN,1130058,NaN,A


In [26]:
# Fill missing values
data['Metascore'] = data['Metascore'].fillna(data['Metascore'].median())

# Split data into features (X) and targets (y)
X = data.drop(['Votes', 'Gross Collection'], axis=1)
y_votes = data['Votes']
y_gross = data['Gross Collection']

# One-hot encode categorical variables
X = pd.get_dummies(X)


In [27]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_votes_train, y_votes_test, y_gross_train, y_gross_test = train_test_split(X, y_votes, y_gross, test_size=0.2, random_state=42)


In [28]:
from xgboost import XGBRegressor

# Initialize and train the XGBoost models for votes and gross collection
model_votes = XGBRegressor()
model_gross = XGBRegressor()

model_votes.fit(X_train, y_votes_train)
model_gross.fit(X_train, y_gross_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [29]:
# Make predictions
y_votes_pred = model_votes.predict(X_test)
y_gross_pred = model_gross.predict(X_test)


In [31]:
from sklearn.metrics import mean_squared_error

# Calculate Mean Squared Error for predictions
mse_votes = mean_squared_error(y_votes_test, y_votes_pred)
mse_gross = mean_squared_error(y_gross_test, y_gross_pred)

print(f"Mean Squared Error for Votes: {mse_votes}")
print(f"Mean Squared Error for Gross Collection: {mse_gross}")


Mean Squared Error for Votes: 148218446375.01172
Mean Squared Error for Gross Collection: 148218446375.01172


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import joblib

# Load the dataset
data = pd.read_csv("merged_data.csv")

# Fill missing values
data['Metascore'] = data['Metascore'].fillna(data['Metascore'].median())

# Split data into features (X) and targets (y)
X = data.drop(['Votes', 'Gross Collection'], axis=1)
y_votes = data['Votes']
y_gross = data['Gross Collection']

# One-hot encode categorical variables
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_votes_train, y_votes_test, y_gross_train, y_gross_test = train_test_split(X, y_votes, y_gross, test_size=0.2, random_state=42)

# Apply feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and tune hyperparameters using GridSearchCV for votes model
param_grid_votes = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

grid_votes = GridSearchCV(XGBRegressor(), param_grid_votes, cv=5, scoring='neg_mean_squared_error')
grid_votes.fit(X_train_scaled, y_votes_train)

# Initialize and tune hyperparameters using GridSearchCV for gross model
param_grid_gross = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'learning_rate': [0.01, 0.1, 0.2]
}

grid_gross = GridSearchCV(XGBRegressor(), param_grid_gross, cv=5, scoring='neg_mean_squared_error')
grid_gross.fit(X_train_scaled, y_gross_train)

# Save the best models
best_model_votes = grid_votes.best_estimator_
best_model_gross = grid_gross.best_estimator_

joblib.dump(best_model_votes, 'best_model_votes.pkl')
joblib.dump(best_model_gross, 'best_model_gross.pkl')

# Explain the basis for choosing the best models
print("Best Model for Votes:")
print(best_model_votes)
print("\nBest Model for Gross Collection:")
print(best_model_gross)


C:\Users\ritul pawar\anaconda3\lib\site-packages\sklearn\utils\extmath.py:985: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\ritul pawar\anaconda3\lib\site-packages\sklearn\utils\extmath.py:990: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
C:\Users\ritul pawar\anaconda3\lib\site-packages\sklearn\utils\extmath.py:1020: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction ** 2 / new_sample_count


Best Model for Votes:
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

Best Model for Gross Collection:
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_cat

    Best Model for Votes:
The output represents the configuration of the best model for predicting the "Votes" using an XGBoost regressor. Let's break down the key components:

      XGBRegressor: This indicates that the model is based on the XGBoost regressor algorithm, which is a powerful gradient boosting technique commonly used for regression tasks.

      max_depth=4: The hyperparameter max_depth specifies the maximum depth of the individual decision trees in the ensemble. In this case, the best model has a maximum tree depth of 4 levels. A moderate depth helps prevent overfitting by limiting the complexity of each tree.

      n_estimators=300: The hyperparameter n_estimators determines the number of trees (individual models) that are combined to form the final prediction. In this best model, a total of 300 trees are used in the ensemble, which provides a balance between model complexity and prediction accuracy.

      learning_rate=0.1: The learning_rate hyperparameter controls the step size at each iteration while optimizing the model. A learning rate of 0.1 indicates a moderate step size, ensuring that the model converges effectively without taking overly large steps that might miss the optimal solution.

      Other Hyperparameters: The output provides various other hyperparameters that are specific to the XGBoost algorithm. These include parameters related to feature subsampling (colsample_bytree), regularization (gamma), and others that influence how the trees are constructed and combined in the ensemble.

    Best Model for Gross Collection:
The output for the "Best Model for Gross Collection" is quite similar, indicating the configuration of the best model for predicting the "Gross Collection" using an XGBoost regressor. The same key components apply:

      XGBRegressor: This indicates the usage of the XGBoost regressor algorithm for predicting the "Gross Collection."

      max_depth=4: Similar to the previous model, the maximum depth of each individual decision tree is set to 4, controlling the complexity of the trees.

      n_estimators=300: Similarly, the number of trees in the ensemble is set to 300, balancing between model complexity and predictive power.

      learning_rate=0.1: The learning rate of 0.1 is maintained here as well, ensuring effective convergence during optimization.

      Other Hyperparameters: Similar to the previous model, various hyperparameters related to tree construction, regularization, and ensemble behavior are provided.

Both models share similar hyperparameter settings, indicating that these values are found to be optimal for the given dataset during hyperparameter tuning and cross-validation. These hyperparameters are chosen to achieve the best possible balance between bias (underfitting) and variance (overfitting), resulting in a well-generalized model that can make accurate predictions on new data.